___

<a href='https://mainacademy.ua/'> <img src='https://mainacademy.ua/wp-content/uploads/2019/03/logo-main.png' alt = 'Хмм, щось з інтернетом'/></a>
___

# Module 4: Work with data professionally!

## Lab work 4



#### Мета: 

* навчитися обробляти дані
* "парсити" сайти

### Завдання 1:

#### Ознайомитися із API Національного банку 
https://old.bank.gov.ua/control/uk/publish/article?art_id=38441973&cat_id=38459171#exchange



1. Підключитися до API.


In [1]:
import requests

url = 'https://bank.gov.ua/NBUStatService/v1/statdirectory/exchange?json'
params = {'valcode' : 'EUR'} 

response = requests.get(url, params=params)
print(response)

<Response [200]>


2. Отримати курс євро за останній рік.

In [2]:
import datetime as DT
import requests

annual_exchange_rate_EURO = {} # ініціалізую словник для подобового значення курсу євро

params = {'date' : '20221018', 'valcode' : 'EUR'} #задамо параметри згідно з документацією API
url = 'https://bank.gov.ua/NBUStatService/v1/statdirectory/exchange?json' # задамо адресу ресурсу
date = DT.datetime.now() # задамо сьогоднішню датау

for day in range(366): # цикл з 365 днів
    params['date'] = (date - DT.timedelta(days=day)).strftime('%Y%m%d') # зміна дати в параметрах запиту
    try:
        response = requests.get(url, params=params) # дані GET запиту
        data = response.json()[0] # запис необхідних даних  у змінну
        annual_exchange_rate_EURO[data['exchangedate']] = data['rate'] # заповнення словника
        start_time += DT.timedelta(days=day) # перезадаємо наступну дату для запиту
    except:
        print('error')
        break

### Працює скрипт через раз, гадаю що проблеми на сервері, тому запишу отримані дані:

In [3]:
# import pickle

# with open('annual_exchange_rate_EURO.bin', 'bw') as f:
#     pickle.dump(annual_exchange_rate_EURO, f)

In [4]:
# зчитую отримані дані:
from io import BytesIO
from urllib.request import urlopen
import pickle

url = 'https://github.com/pruhlo/data-science-assignment/blob/master/assets/annual_exchange_rate_EURO.bin?raw=true'
raw = BytesIO(urlopen(url).read())
EURO = pickle.load(raw)

3. Вивести середнє значення та відхилення курсу за кожний місяць.

In [1]:
import datetime as DT
import numpy as np
import pandas as pd

In [6]:
%%time

month_EURO ={} # ініціалізуємо словник з місячними значеннями курсу євро

for key in EURO.keys():
    month = DT.datetime.strptime(key, '%d.%m.%Y').strftime("%B") # створимо ключ(місяць)
    try:
        month_EURO[month].append(float(EURO[key])) # якщо в словнику є даний список доповнимо його
    except KeyError:
        month_EURO[month] = [float(EURO[key])] # якщо в словнику не має списку створимо його
        
describe_month_EURO = {} # словник для 

for key in month_EURO.keys():
    describe_month_EURO[key + ' mean'] = np.mean(month_EURO[key])
    describe_month_EURO[key + ' std'] = np.std(month_EURO[key])

CPU times: total: 0 ns
Wall time: 9.99 ms


In [7]:
%%time

df = pd.DataFrame.from_dict({'date':EURO.keys(), 'EURO':EURO.values()})
df.date = pd.to_datetime(df.date, format='%d.%m.%Y') # якщо не вказати формат можливо буде помилка при групуванні

result = pd.DataFrame(columns=['mean', 'std'], index=list(range(1,13)))
result['mean'] = df.groupby(df.date.dt.month)['EURO'].mean()
result['std'] = df.groupby(df.date.dt.month)['EURO'].std()

CPU times: total: 15.6 ms
Wall time: 15 ms


In [83]:
# result

In [8]:
%%time

# Ще спосіб групування за допомогою pandas та datetime
DF = pd.DataFrame(columns=['data', 'month', 'EURO'])

for key in EURO:
    DF.loc[len(DF.index)] = [key, DT.datetime.strptime(key, '%d.%m.%Y').strftime("%B"), EURO[key]]

groupBy = DF.groupby('month').mean(numeric_only=True) # numeric_only=True >>> без вказування помилка
groupBy['sdt'] = DF.groupby('month').std(numeric_only=True)

CPU times: total: 797 ms
Wall time: 823 ms


4. Дану інформацію записати у файл за допомогою pickle.

In [9]:
import pickle

with open('mydata_LW-4.bin', 'bw') as f:
    pickle.dump(month_EURO, f)
    pickle.dump(describe_month_EURO, f)
    pickle.dump(df, f)
    pickle.dump(result, f)
    pickle.dump(DF, f)
    pickle.dump(groupBy, f)

In [29]:
# ПРОЧИТАТИ з репозиторію вказані дані
# url = 'https://github.com/pruhlo/data-science-assignment/blob/master/assets/mydata_LW-4.bin?raw=true'
# raw = BytesIO(urlopen(url).read())
# month_EURO1 = pickle.load(raw)
# describe_month_EURO1 = pickle.load(raw)
# df1 = pickle.load(raw)
# result1 = pickle.load(raw)
# DF1 = pickle.load(raw)
# groupBy1 = pickle.load(raw)

### Завдання 2:

Потрібно проаналізувати всі товари на сайті: 
https://smallpacking.agrosem.ua/shop/

Створіть базу даних, що містить таблицю із такими стовбцями: 
    - артикул товару
    - найменування
    - ціна 
    - вага

In [16]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

def make_req(url, page):
    
    ''' Query with nuber of page into web store
    HEADERS see: https://raw.githubusercontent.com/Lidkins/DS/915bdbd30ca3b0dd2fc507db94b41ee62cbcebf3/Lab%20work%204.ipynb'''
#     HEADERS = {
#              'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:49.0) Gecko/20100101 Firefox/49.0',
#              'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
#              'Accept-Language': 'ua-UA,en;q=0.5',
#              'Accept-Encoding': 'gzip, deflate',
#              'DNT': '1',
#              'Connection': 'keep-alive',
#              'Upgrade-Insecure-Requests': '1',
#              'page':f'{page}'
    HEADERS = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36',
    }
#     headers = {
#         'User-Agent': 'Mozilla/5.0',
#         'page':f'{page}'}
    resp = requests.get(url, headers=HEADERS, params = {'page': f'{page}'})
    return resp

def get_data(df):
    base_select_tags = 'div.item' # базовий тег для отримання даних
    for page in range(1,30):
        url = 'https://smallpacking.agrosem.ua/' # URLs сторінок з данними
        response = make_req(url, page) # результат запиту
        soup = BeautifulSoup(response.text, 'html.parser') # об'єкт супу
        list_tags = [i.get('content') for i in soup.select(f'{base_select_tags}')] # витягаємо контент
        for _ in list_tags:
            dicty = eval(_) # отримуємо словник
            df.loc[len(df.index)] = dicty # додаємо рядок до датафрейму
    return df

df = pd.DataFrame(columns=['id', 'name', 'weight', 'price'])

df = get_data(df)
df

,id,name,weight,price
0,1604,ДОБРИВО ДЛЯ ОРХІДЕЙ,180 г,119.00
1,1609,ДОБРИВО ДЛЯ ТРОЯНД ТА КВІТУЧИХ РОСЛИН,180 г,119.00
2,1600,ДОБРИВО ДЛЯ КВІТУЧИХ КІМНАТНИХ РОСЛИН,180 г,119.00
3,1599,ДОБРИВО ДЛЯ ДЕКОРАТИВНО-ЛИСТЯНИХ КІМНАТНИХ РОСЛИН,180 г,119.00
4,1522,ДОБРИВО ДЛЯ ДЕКОРАТИВНИХ ЛИСТОВИХ,3 кг,499.00
...,...,...,...,...
1039,1600,ДОБРИВО ДЛЯ КВІТУЧИХ КІМНАТНИХ РОСЛИН,180 г,119.00
1040,1599,ДОБРИВО ДЛЯ ДЕКОРАТИВНО-ЛИСТЯНИХ КІМНАТНИХ РОСЛИН,180 г,119.00
1041,1608,ДОБРИВО ДЛЯ САМШИТУ ТА ВІЧНОЗЕЛЕНИХ ЧАГАРНИКІВ,180 г,119.00
1042,1610,ДОБРИВО ДЛЯ ХВОЙНИХ РОСЛИН,180 г,129.00


In [18]:
df.id.nunique()

26

### Завдання 3: 

Ознайомтеся із роботою SQLite та відповідним модулем у Python.
Завантажте базу даних для виконання лабораторної роботи.
Підключіться до завантаженої бази SQLite.

1. Виведіть інформацію про дану базу.

In [74]:
# Спробую працювати з БД завантаженої до github
from io import BytesIO as Memory
import requests
import sqlite3
import pandas as pd
from IPython.display import display, HTML

def download_file_to_obj(url, file_obj):
    """
    See: https://stackoverflow.com/questions/69522022/open-sqlite-database-from-http-in-memory
    """
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        for chunk in r.iter_content(chunk_size=None):
            if chunk:
                file_obj.write(chunk)

def query(SQL_query):
    source_url = 'https://github.com/pruhlo/data-science-assignment/blob/master/tysql.sqlite?raw=true'
    db_in_mem = Memory()
    print('Downloading..')
    download_file_to_obj(source_url, db_in_mem)
    print('Complete!')
    try:
        with sqlite3.connect(":memory:") as con:
            df = pd.read_sql_query(f"{SQL_query}", con)
            display(HTML(df.to_html()))
    except Error as e:
        print('Error query...')
        print(e)

query('PRAGMA database_list;') # https://stackoverflow.com/questions/59709946/sqlite3-operationalerror-near-show-syntax-error

Downloading..
Complete!


,seq,name,file
0,0,main,


2. Виведіть перелік всіх таблиць.

In [77]:
query("PRAGMA table_list;") 

Downloading..
Complete!


,schema,name,type,ncol,wr,strict
0,main,sqlite_schema,table,5,0,0
1,temp,sqlite_temp_schema,table,5,0,0


In [104]:
# Всіх таблиць не видно, буду працювати з БД з диску
def query(SQL_query):
    try:
        with sqlite3.connect("tysql.sqlite") as con:
            df = pd.read_sql_query(f"{SQL_query}", con)
            display(HTML(df.to_html()))
    except Error as e:
        print(e)

query("PRAGMA table_list;")

,schema,name,type,ncol,wr,strict
0,main,Vendors,table,7,0,0
1,main,Products,table,5,0,0
2,main,Orders,table,3,0,0
3,main,OrderItems,table,5,0,0
4,main,Customers,table,9,0,0
5,main,sqlite_schema,table,5,0,0
6,temp,sqlite_temp_schema,table,5,0,0


3. Список всіх cust_id з таблиці Customers table.

In [81]:
query('SELECT cust_id FROM Customers')

,cust_id
0,1000000001
1,1000000002
2,1000000003
3,1000000004
4,1000000005


4. Всю таблицю Customers table

In [82]:
query('SELECT * FROM Customers;')

,cust_id,cust_name,cust_address,cust_city,cust_state,cust_zip,cust_country,cust_contact,cust_email
0,1000000001,Village Toys,200 Maple Lane,Detroit,MI,44444,USA,John Smith,sales@villagetoys.com
1,1000000002,Kids Place,333 South Lake Drive,Columbus,OH,43333,USA,Michelle Green,None
2,1000000003,Fun4All,1 Sunny Place,Muncie,IN,42222,USA,Jim Jones,jjones@fun4all.com
3,1000000004,Fun4All,829 Riverside Drive,Phoenix,AZ,88888,USA,Denise L. Stephens,dstephens@fun4all.com
4,1000000005,The Toy Store,4545 53rd Street,Chicago,IL,54545,USA,Kim Howard,None


5. Список клієнтів (cust_names) відсортованих від Z до A

In [83]:
query('SELECT * FROM Customers ORDER BY cust_name DESC')

,cust_id,cust_name,cust_address,cust_city,cust_state,cust_zip,cust_country,cust_contact,cust_email
0,1000000001,Village Toys,200 Maple Lane,Detroit,MI,44444,USA,John Smith,sales@villagetoys.com
1,1000000005,The Toy Store,4545 53rd Street,Chicago,IL,54545,USA,Kim Howard,None
2,1000000002,Kids Place,333 South Lake Drive,Columbus,OH,43333,USA,Michelle Green,None
3,1000000003,Fun4All,1 Sunny Place,Muncie,IN,42222,USA,Jim Jones,jjones@fun4all.com
4,1000000004,Fun4All,829 Riverside Drive,Phoenix,AZ,88888,USA,Denise L. Stephens,dstephens@fun4all.com


6. Таблицю клієнтів та замовлень (cust_id і order_num) . Відсортуйте по клієнту і потім по даті замовлення

In [84]:
query('SELECT cust_id, order_num FROM Orders ORDER BY cust_id, order_date')

,cust_id,order_num
0,1000000001,20009
1,1000000001,20005
2,1000000003,20006
3,1000000004,20007
4,1000000005,20008


7. Таблицю (на основі Items) з кількість та вартістю товару. Відсортуйте в порядку спадання по кількості та вартості

In [85]:
query('SELECT * FROM OrderItems ORDER BY quantity DESC, item_price DESC')

,order_num,order_item,prod_id,quantity,item_price
0,20009,1,BNBG01,250,2.49
1,20009,2,BNBG02,250,2.49
2,20009,3,BNBG03,250,2.49
3,20005,2,BR03,100,10.99
4,20005,1,BR01,100,5.49
5,20007,2,BNBG01,100,2.99
6,20007,3,BNBG02,100,2.99
7,20007,4,BNBG03,100,2.99
8,20007,1,BR03,50,11.49
9,20007,5,RGAN01,50,4.49


8. Товар (з таблиці Products), ціна якого становить 9.49

In [86]:
query('SELECT * FROM Products WHERE prod_price = 9.49')

,prod_id,vend_id,prod_name,prod_price,prod_desc
0,RYL01,FNG01,King doll,9.49,12 inch king doll with royal garments and crown
1,RYL02,FNG01,Queen doll,9.49,12 inch queen doll with royal garments and crown


9. Виведіть назву товару та ціну, яка лежить в діапазоні від  3 до 6. Відсортуйте результат в по ціні в порядку зростання

In [87]:
query('''SELECT prod_name, prod_price 
FROM Products 
WHERE (prod_price >= 3) and (prod_price <= 6) 
ORDER BY prod_price''')

,prod_name,prod_price
0,Fish bean bag toy,3.49
1,Bird bean bag toy,3.49
2,Rabbit bean bag toy,3.49
3,Raggedy Ann,4.99
4,8 inch teddy bear,5.99


10. Кількість товару, що було продано

In [88]:
query('SELECT SUM(quantity) as TOTAL_item FROM OrderItems')

,TOTAL_item
0,1430


11. Кількість найменувань товару, ціна якого більша за 4

In [89]:
query('SELECT COUNT(prod_name) FROM Products WHERE prod_price >= 4')

,COUNT(prod_name)
0,6


12. Розробіть алгоритм для виведення 3 найдорожчих товарів в базі

In [90]:
query('SELECT prod_name, prod_price FROM Products ORDER BY prod_price DESC LIMIT 3;')

,prod_name,prod_price
0,18 inch teddy bear,11.99
1,King doll,9.49
2,Queen doll,9.49


13. Підрахуйте кількість замовлень для кожного клієнта використовуючи підзапити

In [92]:
query("""SELECT cust_name, count(order_num)
                FROM Orders
                LEFT JOIN Customers ON Orders.cust_id == Customers.cust_id
                GROUP BY cust_name""")

,cust_name,count(order_num)
0,Fun4All,2
1,The Toy Store,1
2,Village Toys,2


14. Виведіть список клієнтів та їх замовлення (використайте JOIN)

In [91]:
query("""SELECT cust_name, order_num 
                FROM Customers JOIN Orders ON Customers.cust_id == Orders.cust_id
                ORDER BY cust_name""")

,cust_name,order_num
0,Fun4All,20006
1,Fun4All,20007
2,The Toy Store,20008
3,Village Toys,20005
4,Village Toys,20009


15. Зробіть два запити: список товарів, ціна яких менша 5 та список товарів, ціна який більша рівна 5. Використайте UNION для об'єднання цих запитів

In [105]:
query("""  SELECT prod_price, prod_name
        FROM Products
        WHERE prod_price < 5
        
        UNION 
        
        SELECT prod_price, prod_name
        FROM Products
        WHERE prod_price >= 5""")

,prod_price,prod_name
0,3.49,Bird bean bag toy
1,3.49,Fish bean bag toy
2,3.49,Rabbit bean bag toy
3,4.99,Raggedy Ann
4,5.99,8 inch teddy bear
5,8.99,12 inch teddy bear
6,9.49,King doll
7,9.49,Queen doll
8,11.99,18 inch teddy bear


### Завдання 4: 

Зареєструйтеся на сайті  http://www.sql-ex.ru/

Виконайте завдання із блоку SELECT; з 1 по 30 завдання

Код запитів вставляйте сюди


##### 1 Знайдіть номер моделі, швидкість та розмір жорсткого диска для всіх ПК вартістю менше 500 дол. Вивести: model, speed та hd
```MySQL
SELECT model, speed, hd
FROM PC
WHERE price < 500
```

##### 2 Знайдіть виробників принтерів. Вивести: maker
```MySQL
SELECT DISTINCT maker
FROM Product
WHERE type='Printer'
```

##### 3 Знайдіть номер моделі, обсяг пам'яті та розміри екранів ПК-блокнотів, ціна яких перевищує 1000 дол.
```MySQL
SELECT model, ram, screen
FROM Laptop
WHERE price > 1000
```

#####  4 Знайдіть усі записи таблиці Printer для кольорових принтерів.
```MySQL
SELECT *
FROM Printer
WHERE color='y'
```

##### 5 Знайдіть номер моделі, швидкість та розмір жорсткого диска ПК, що мають 12x або 24x CD та ціну менше 600 дол.
```MySQL
SELECT model, speed, hd
FROM PC
WHERE (cd in ('12x','24x')) and (price < 600)
```

##### 6 Для кожного виробника, що випускає ноутбуки з об'ємом жорсткого диска щонайменше 10 Гбайт, знайти швидкості таких ноутбуків. Вивести: виробника, швидкість.
```MySQL
SELECT DISTINCT maker, speed
FROM Product JOIN Laptop ON Laptop.model = Product.model
WHERE (hd >=10)
```

#####  7 Знайдіть номери моделей та ціни всіх наявних у продажу продуктів (будь-якого типу) виробника B (латинська літера).
### пройшов тест на основній базі
```MySQL
SELECT DISTINCT Product.model, 
 ISNULL(PC.price, 0)+ISNULL(Laptop.price, 0)+ISNULL(Printer.price, 0) AS price
FROM (((Product LEFT JOIN  PC ON Product.model = PC.model) LEFT JOIN 
 Laptop ON Product.model = Laptop.model) LEFT JOIN 
 Printer ON Product.model = Printer.model)
WHERE Product.maker = 'B'
```

##### 8 Знайдіть виробника, що випускає ПК, але не ноутбуки.
```MySQL
SELECT DISTINCT maker
FROM Product
WHERE type = 'PC' AND 
      maker NOT IN (SELECT maker 
                    FROM Product 
                    WHERE type = 'Laptop')
```

#####  9 Знайдіть виробників ПК із процесором не менше 450 МГц. Вивести: Maker
### пройшов тест на основній базі
```MySQL
SELECT DISTINCT maker
FROM Product INNER JOIN PC ON Product.model = PC.model
WHERE speed > 450 AND type='PC'
```

##### 10 Знайдіть моделі принтерів, що мають найвищу ціну. Вивести: model, price
```MySQL
SELECT model, price
FROM Printer
WHERE price = (SELECT MAX(price) FROM Printer)
```

##### 11 Знайдіть середню швидкість ПК.
```MySQL
SELECT AVG(speed) as Avg_speed
FROM PC
```

##### 12 Знайдіть середню швидкість ноутбуків, ціна яких перевищує 1000 дол.
```MySQL
SELECT AVG(speed) AS Avg_speed  
FROM Laptop 
WHERE price > 1000
```

##### 13 Найдите среднюю скорость ПК, выпущенных производителем A.
```MySQL
SELECT AVG(speed) AS Avg_speed
FROM Product JOIN PC ON Product.model=PC.model
WHERE maker = 'A'
```

#####  14 Знайдіть клас, ім'я та країну для кораблів із таблиці Ships, що мають не менше 10 гармат.
```MySQL
SELECT Ships.class, name, country
FROM Ships JOIN Classes ON Ships.class = Classes.class
WHERE numGuns>=10;
```

##### 15 Знайдіть розміри жорстких дисків, що збігаються у двох і більше ПК. Вивести: HD
```MySQL
SELECT hd 
FROM PC 
GROUP BY hd 
HAVING COUNT(hd)>=2
```

##### 16 Знайдіть пару моделей PC, що мають однакову швидкість і RAM. У результаті кожна пара вказується лише один раз, тобто (i,j), але не (j,i), порядок виведення: модель з більшим номером, модель з меншим номером, швидкість та RAM.
### списав
```MySQL
SELECT MAX(model1), MIN(model2), MAX(speed), MAX(ram) 
FROM (SELECT pc1.model AS model1, pc2.model AS model2, pc1.speed, pc2.ram, 
             CASE WHEN CAST(pc1.model AS NUMERIC(6,2)) > 
                       CAST(pc2.model AS NUMERIC(6,2)) 
                  THEN pc1.model+pc2.model  
                  ELSE pc2.model+pc1.model  
             END AS sm 
      FROM PC pc1, PC pc2 
      WHERE pc1.speed = pc2.speed AND 
            pc1.ram = pc2.ram AND 
            pc1.model <> pc2.model
      ) a 
GROUP BY a.sm
```

##### 17 Знайдіть моделі ноутбуків, швидкість яких менша за швидкість кожного з ПК. Вивести: type, model, speed
```MySQL
SELECT DISTINCT type, Laptop.model, speed
FROM Laptop, Product
WHERE Product.model = Laptop.model AND
      Laptop.speed < (SELECT MIN(speed) FROM PC)
```

##### 18 Знайдіть виробників найдешевших кольорових принтерів. Вивести: maker, price
### пройшов тест на основній базі
```MySQL
select maker, price
from printer, product
WHERE product.model = printer.model and price = (select min(price) from printer where color='y') and color='y'
```

##### 19 Для кожного виробника, що має моделі в таблиці Laptop, знайдіть середній розмір екрану ноутбуків, що випускаються ним. Вивести: maker, середній розмір екрану.
```MySQL
SELECT maker, avg(screen)
FROM laptop, product
WHERE laptop.model=product.model
GROUP BY maker
```

#####  20 Знайдіть виробників, що випускають щонайменше три різні моделі ПК. Вивести: Maker, кількість моделей ПК.
```MySQL
SELECT maker, count(model) as Count_Model
FROM Product
WHERE type='PC'
GROUP BY maker
HAVING count(model)>=3
```

##### 21 Знайдіть максимальну ціну ПК, що випускаються кожним виробником, який має моделі в таблиці PC. Вивести: maker, максимальна ціна.
```MySQL
SELECT maker, max(price)
FROM Product, PC
WHERE type='PC' and PC.model=Product.model
GROUP BY maker
```

#####  22 Для кожного значення швидкості ПК, що перевищує 600 МГц, визначте середню ціну ПК із такою самою швидкістю. Вивести: speed, середня ціна.
```MySQL
SELECT speed, AVG(price)
FROM PC
WHERE speed > 600
GROUP BY speed
```
##### 23 Знайдіть виробників, які виробляли б як ПК зі швидкістю щонайменше 750 МГц, і ноутбуки зі швидкістю щонайменше 750 МГц. Вивести: Maker
### списав
```MySQL
SELECT DISTINCT maker
FROM Product
WHERE maker IN (
    SELECT DISTINCT maker 
    FROM Product JOIN PC ON Product.model = PC.model
    WHERE speed >= 750
)
AND maker IN (
    SELECT DISTINCT maker 
    FROM Product JOIN Laptop ON Product.model = Laptop.model
    WHERE speed >= 750
)
```

##### 24 Перерахуйте номери моделей будь-яких типів, що мають найвищу ціну по всій наявній у базі даних продукції.
```MySQL
WITH pivot_table AS (SELECT model, price
                     FROM (SELECT product.model as model, ISNULL(PC.price, 0)+ISNULL(Laptop.price,
                                                             0)+ISNULL(Printer.price, 0) AS price
                           FROM (((Product LEFT JOIN PC ON Product.model = PC.model) LEFT JOIN 
                                     Laptop ON Product.model = Laptop.model) LEFT JOIN 
                                     Printer ON Product.model = Printer.model)) as tables)
SELECT model
FROM pivot_table
WHERE price = (SELECT MAX(price) FROM pivot_table)
```

##### 25 Знайдіть виробників принтерів, які виробляють ПК з найменшим об'ємом RAM і найшвидшим процесором серед усіх ПК, що мають найменший об'єм RAM. Вивести: Maker
```MySQL
WITH pivot_table AS
(SELECT Product.maker, ram, speed, Product.type
FROM ((Product LEFT JOIN PC ON Product.model = PC.model) LEFT JOIN Printer ON Product.model = Printer.model))

SELECT DISTINCT maker
FROM pivot_table
WHERE speed = (SELECT MAX(speed) FROM pivot_table WHERE ram = (SELECT min(ram) FROM pivot_table)) and maker IN (SELECT maker FROM pivot_table WHERE type = 'Printer') and ram = (SELECT min(ram) FROM pivot_table)
```

##### 26 Знайдіть середню ціну ПК та ПК-блокнотів, випущених виробником A (латинська літера). Вивести: одна загальна середня ціна.
```MySQL
SELECT AVG(price)
FROM (SELECT price, model, speed, ram, hd
      FROM PC 
      WHERE model IN (SELECT model FROM Product WHERE maker='A' AND type ='PC')
      UNION
      SELECT price, model, speed, ram, hd
      FROM Laptop
      WHERE model IN (SELECT model FROM Product WHERE maker='A' AND type ='Laptop')) as p
   ```

##### 27 Знайдіть середній розмір диска ПК кожного з виробників, які випускають і принтери. Вивести: maker, середній розмір HD.

```MySQL
SELECT maker, AVG(hd)
FROM (SELECT maker, hd FROM PC JOIN Product ON  PC.model=Product.model WHERE maker IN (SELECT maker FROM Printer JOIN Product ON Printer.model=Product.model)) as t
GROUP BY maker
```

##### 28 Використовуючи таблицю Product, визначити кількість виробників, що випускають за однією моделлю.
```MySQL
SELECT COUNT(*) as count 
FROM (SELECT count(model) as cnt
FROM Product
GROUP BY maker
HAVING count(model) = 1) as t
WHERE cnt=1
```

##### 29 У припущенні, що прихід та витрата грошей на кожному пункті прийому фіксується не частіше ніж один раз на день [тобто. первинний ключ (пункт, дата)], написати запит із вихідними даними (пункт, дата, прихід, витрата). Використовувати таблиці Income_o та Outcome_o.
### списав
```MySQL
SELECT i.point, i.date, i.inc, o.out
FROM Income_o i LEFT JOIN Outcome_o o ON i.point = o.point AND i.date = o.date

UNION

SELECT o.point, o.date, i.inc, o.out
FROM Outcome_o o LEFT JOIN Income_o i ON o.point = i.point AND o.date = i.date
```

##### 30 В предположении, что приход и расход денег на каждом пункте приема фиксируется произвольное число раз (первичным ключом в таблицах является столбец code), требуется получить таблицу, в которой каждому пункту за каждую дату выполнения операций будет соответствовать одна строка.
### списав
```MySQL
SELECT point, date, SUM(sum_out), SUM(sum_inc)
FROM (
SELECT point, date, SUM(inc) as sum_inc, null as sum_out
FROM  Income
GROUP BY point, date

UNION

SELECT point, date, null as sum_inc, SUM(out) as sum_out
FROM  Outcome
GROUP BY point, date
) as T
GROUP BY point, date
ORDER BY point
```